In [1]:
from ogb.nodeproppred import PygNodePropPredDataset


In [2]:
dataset = PygNodePropPredDataset(name='ogbn-mag')

/home/accl-ros/anaconda3/envs/myenv/lib/python3.12/site-packages/ogb/nodeproppred/dataset_pyg.py:69: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.data, self.slices = t

In [3]:
data=dataset[0]

In [43]:
# Clear outputs in-place to shrink notebook size
!jupyter nbconvert --clear-output --inplace "Heterogenous graph transformer.ipynb"

usage: jupyter [-h] [--version] [--config-dir] [--data-dir] [--runtime-dir]
               [--paths] [--json] [--debug]
               [subcommand]

Jupyter: Interactive Computing

positional arguments:
  subcommand     the subcommand to launch

options:
  -h, --help     show this help message and exit
  --version      show the versions of core jupyter packages and exit
  --config-dir   show Jupyter config dir
  --data-dir     show Jupyter data dir
  --runtime-dir  show Jupyter runtime dir
  --paths        show all Jupyter paths. Add --json for machine-readable
                 format.
  --json         output paths as machine-readable json
  --debug        output debug information about paths

Available subcommands: kernel kernelspec migrate run troubleshoot trust

Jupyter command `jupyter-nbconvert` not found.


In [5]:
edge_index_dict = data.edge_index_dict
edge_index_dict

{('author',
  'affiliated_with',
  'institution'): tensor([[      0,       1,       2,  ..., 1134645, 1134647, 1134648],
         [    845,     996,    3197,  ...,    5189,    4668,    4668]]),
 ('author',
  'writes',
  'paper'): tensor([[      0,       0,       0,  ..., 1134647, 1134648, 1134648],
         [  19703,  289285,  311768,  ...,  657395,  671118,  719594]]),
 ('paper',
  'cites',
  'paper'): tensor([[     0,      0,      0,  ..., 736388, 736388, 736388],
         [    88,  27449, 121051,  ..., 421711, 427339, 439864]]),
 ('paper',
  'has_topic',
  'field_of_study'): tensor([[     0,      0,      0,  ..., 736388, 736388, 736388],
         [   145,   2215,   3205,  ...,  21458,  22283,  31934]])}

In [6]:
import json, os
import math, copy, time
import numpy as np
from collections import defaultdict
import pandas as pd

In [7]:
user1 = {'type': 'User', 'id': 'U1'}
user2 = {'type': 'User', 'id': 'U2'}
post1 = {'type': 'Post', 'id': 'P1'}

In [8]:
class Graph():
    def __init__(self):
        super(Graph, self).__init__()
        self.node_forward = defaultdict(lambda:{}) # {node1: {node2: edge1, node3: edge2}}
        self.node_bacward = defaultdict(lambda:[]) # {node1: [node2, node3]}

        self.node_features = defaultdict(lambda:[]) # {node1: [feat1, feat2]}
        self.edge_list = defaultdict( #target_type
                            lambda: defaultdict(  #source_type
                                lambda: defaultdict(  #relation_type
                                    lambda: defaultdict(  #target_id
                                        lambda: defaultdict( #source_id(
                                            lambda: int # time
                                        ))))) #
        self.times = {} # {time: {edge_index: time}}
    def add_node(self,node):
        nfl=self.node_forward[node['type']] # how? why? 
        if node['id'] not in nfl:
            self.node_bacward[node['type']] += [node]
            ser = len(nfl)
            nfl[node['id']] = ser
            return ser
        return nfl[node['id']]
    def add_edge(self, source_node, target_node, time = None, relation_type = None, directed = True):
        edge = [self.add_node(source_node), self.add_node(target_node)]
        '''
            Add bi-directional edges with different relation type
        '''
        self.edge_list[target_node['type']][source_node['type']][relation_type][edge[1]][edge[0]] = time
        if directed:
            self.edge_list[source_node['type']][target_node['type']]['rev_' + relation_type][edge[0]][edge[1]] = time
        else:
            self.edge_list[source_node['type']][target_node['type']][relation_type][edge[0]][edge[1]] = time
        self.times[time] = True
    def update_node(self, node):
        nbl = self.node_bacward[node['type']]
        ser = self.add_node(node)
        for k in node:
            if k not in nbl[ser]:
                nbl[ser][k] = node[k]
    def get_meta_graph(self):
        #types = self.get_types()
        metas = []
        for target_type in self.edge_list:
            for source_type in self.edge_list[target_type]:
                for r_type in self.edge_list[target_type][source_type]:
                    metas += [(target_type, source_type, r_type)]
        return metas
    def get_types(self):
        return list(self.node_feature.keys())





In [9]:
graph=Graph()
graph.add_node(user1)
graph.add_node(user2)
# Step 1: Add initial nodes
user_node = {'type': 'User', 'id': 'U1', 'name': 'Alice'}
post_node = {'type': 'Post', 'id': 'P1', 'title': 'Graph Theory'}
graph.add_edge(source_node=user_node, target_node=post_node, time=1, relation_type='authored', directed=True)

graph.edge_list

defaultdict(<function __main__.Graph.__init__.<locals>.<lambda>()>,
            {'Post': defaultdict(<function __main__.Graph.__init__.<locals>.<lambda>.<locals>.<lambda>()>,
                         {'User': defaultdict(<function __main__.Graph.__init__.<locals>.<lambda>.<locals>.<lambda>.<locals>.<lambda>()>,
                                      {'authored': defaultdict(<function __main__.Graph.__init__.<locals>.<lambda>.<locals>.<lambda>.<locals>.<lambda>.<locals>.<lambda>()>,
                                                   {0: defaultdict(<function __main__.Graph.__init__.<locals>.<lambda>.<locals>.<lambda>.<locals>.<lambda>.<locals>.<lambda>.<locals>.<lambda>()>,
                                                                {0: 1})})})}),
             'User': defaultdict(<function __main__.Graph.__init__.<locals>.<lambda>.<locals>.<lambda>()>,
                         {'Post': defaultdict(<function __main__.Graph.__init__.<locals>.<lambda>.<locals>.<lambda>.<locals>.<lambda>

In [10]:
graph.get_meta_graph()

[('Post', 'User', 'authored'), ('User', 'Post', 'rev_authored')]

In [11]:
updated_user_node = {'type': 'User', 'id': 'U1', 'age': 30}
graph.add_node(updated_user_node)

0

In [12]:
graph.node_forward

defaultdict(<function __main__.Graph.__init__.<locals>.<lambda>()>,
            {'User': {'U1': 0, 'U2': 1}, 'Post': {'P1': 0}})

In [13]:
graph.node_bacward['User'][0]['ppp'] = 1

In [14]:
years=data.node_year['paper'].t().numpy()[0]

In [15]:
for k in updated_user_node:
    print(k)

type
id
age


In [16]:
years = data.node_year['paper'].t().numpy()[0]

In [17]:
edg   = graph.edge_list

In [19]:
edg   = graph.edge_list

In [20]:
for key in data.edge_index_dict:
    print(key)
    edges = edge_index_dict[key]
    s_type, r_type, t_type = key[0], key[1], key[2]
    e_list = edg[t_type][s_type][r_type]
    r_list = edg[s_type][t_type]['rev_' + r_type]
    for s_id, t_id in edges.t().tolist():
        year=None
        if s_type == 'paper':
            year = years[s_id]
        elif t_type == 'paper':
            year = years[t_id]
        e_list[s_id][t_id] = year
        r_list[t_id][s_id] = year
        




('author', 'affiliated_with', 'institution')
('author', 'writes', 'paper')
('paper', 'cites', 'paper')
('paper', 'has_topic', 'field_of_study')


In [37]:
# --- STEP 1: Accurate Node Counts ---
all_node_types = sorted(list(all_types_in_edges))
node_type_to_id = {ntype: i for i, ntype in enumerate(all_node_types)}

x_list = []
type_list = []
offsets = {}
curr_offset = 0

for ntype in all_node_types:
    # 1. Get the actual count of nodes for this type
    # We check your graph.node_forward map first
    num_nodes = len(graph.node_forward.get(ntype, {}))
    
    # 2. Safety check: If node_forward is empty, check the data dictionary
    if num_nodes == 0 and ntype in data.num_nodes_dict:
        num_nodes = data.num_nodes_dict[ntype]
    
    offsets[ntype] = curr_offset
    
    # 3. Create/Fetch features
    if ntype == 'paper' and 'paper' in data.x_dict:
        feats = data.x_dict['paper']
    else:
        # Every node MUST have a feature vector of the same width (128)
        feats = torch.zeros((num_nodes, 128))
    
    # --- DEBUG PRINTS ---
    # This helps you see if a type has 0 nodes, which might cause the error
    print(f"Type: {ntype:15} | Nodes: {num_nodes:8} | Offset: {curr_offset:8}")
    
    x_list.append(feats)
    # CRITICAL: type_list must have 'num_nodes' entries for THIS specific type
    type_list.append(torch.full((num_nodes,), node_type_to_id[ntype], dtype=torch.long))
    
    curr_offset += num_nodes

# --- STEP 2: Final Tensors ---
x_all = torch.cat(x_list, dim=0).to(device)
node_type = torch.cat(type_list, dim=0).to(device)

# --- HUMAN CHECK ---
# These two MUST be the same number. If they aren't, the mask will fail.
print(f"\nFinal Check:")
print(f"Total Features Rows: {x_all.size(0)}")
print(f"Total Node Type Rows: {node_type.size(0)}")

if x_all.size(0) != node_type.size(0):
    print("CRITICAL ERROR: Sizes still don't match! Check your num_nodes logic.")

Type: Post            | Nodes:        1 | Offset:        0
Type: User            | Nodes:        2 | Offset:        1
Type: author          | Nodes:  1134649 | Offset:        3
Type: field_of_study  | Nodes:    59965 | Offset:  1134652
Type: institution     | Nodes:     8740 | Offset:  1194617
Type: paper           | Nodes:   736389 | Offset:  1203357

Final Check:
Total Features Rows: 1939746
Total Node Type Rows: 1939746


In [38]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [40]:
import numpy as np

# 1. Select 10% of the Paper nodes
paper_type_id = node_type_to_id['paper']
all_paper_indices = torch.where(node_type == paper_type_id)[0]

# Shuffle and pick 10%
num_papers = len(all_paper_indices)
sample_size = int(num_papers * 0.10)
perm = torch.randperm(num_papers)
sampled_paper_indices = all_paper_indices[perm[:sample_size]]

# 2. Filter edges
# We only keep edges where the destination (target) is one of our sampled papers
# or the source is one of our sampled papers.
mask = torch.isin(edge_index_all[0], sampled_paper_indices) | torch.isin(edge_index_all[1], sampled_paper_indices)

sampled_edge_index = edge_index_all[:, mask]
sampled_edge_type = edge_type_all[mask]

# 3. Identify all unique nodes involved in these 10% edges
relevant_nodes = torch.unique(sampled_edge_index)

# 4. Create a mapping to shrink the node IDs (re-indexing)
# This prevents the model from trying to allocate a 1.9M row matrix
new_x = x_all[relevant_nodes]
new_node_type = node_type[relevant_nodes]

# Re-map the edge indices to the new smaller node set [0, len(relevant_nodes)]
n_id_map = torch.full((x_all.size(0),), -1, device=device)
n_id_map[relevant_nodes] = torch.arange(relevant_nodes.size(0), device=device)

final_edge_index = n_id_map[sampled_edge_index]
final_edge_type = sampled_edge_type

print(f"Original Nodes: {x_all.size(0)} | Sampled Nodes: {new_x.size(0)}")
print(f"Original Edges: {edge_index_all.size(1)} | Sampled Edges: {final_edge_index.size(1)}")

Original Nodes: 1939746 | Sampled Nodes: 0
Original Edges: 42222016 | Sampled Edges: 0


In [41]:
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.utils import softmax
from torch_scatter import scatter_add

class ScratchHGTConv(nn.Module):
    def __init__(self, in_dim, out_dim, num_types, num_relations, n_heads=4):
        super().__init__()
        self.out_dim, self.n_heads = out_dim, n_heads
        self.d_k = out_dim // n_heads
        self.sqrt_dk = self.d_k ** 0.5
        
        self.k_linears = nn.ModuleList([nn.Linear(in_dim, out_dim) for _ in range(num_types)])
        self.q_linears = nn.ModuleList([nn.Linear(in_dim, out_dim) for _ in range(num_types)])
        self.v_linears = nn.ModuleList([nn.Linear(in_dim, out_dim) for _ in range(num_types)])
        self.a_linears = nn.ModuleList([nn.Linear(out_dim, out_dim) for _ in range(num_types)])
        self.norms = nn.ModuleList([nn.LayerNorm(out_dim) for _ in range(num_types)])
        
        self.relation_att = nn.Parameter(torch.Tensor(num_relations, n_heads, self.d_k, self.d_k))
        self.relation_msg = nn.Parameter(torch.Tensor(num_relations, n_heads, self.d_k, self.d_k))
        nn.init.xavier_uniform_(self.relation_att); nn.init.xavier_uniform_(self.relation_msg)

    def forward(self, node_feat, node_type, edge_index, edge_type):
        src, dst = edge_index
        N = node_feat.size(0)
        
        k_all = torch.zeros(N, self.out_dim, device=node_feat.device)
        q_all = torch.zeros(N, self.out_dim, device=node_feat.device)
        v_all = torch.zeros(N, self.out_dim, device=node_feat.device)
        
        for t in range(len(self.k_linears)):
            mask = (node_type == t)
            if mask.any():
                k_all[mask] = self.k_linears[t](node_feat[mask])
                q_all[mask] = self.q_linears[t](node_feat[mask])
                v_all[mask] = self.v_linears[t](node_feat[mask])

        k, q, v = [x.view(-1, self.n_heads, self.d_k) for x in [k_all, q_all, v_all]]

        # Attention calculation
        k_rel = torch.einsum('ehd,ehdf->ehf', k[src], self.relation_att[edge_type])
        att_score = (q[dst] * k_rel).sum(dim=-1) * (1.0 / self.sqrt_dk)
        att_weight = softmax(att_score, dst, num_nodes=N)
        
        # Message calculation
        v_rel = torch.einsum('ehd,ehdf->ehf', v[src], self.relation_msg[edge_type])
        messages = (v_rel * att_weight.unsqueeze(-1)).reshape(-1, self.out_dim)
        
        aggr_out = scatter_add(messages, dst, dim=0, dim_size=N)
        
        final_out = aggr_out.clone()
        for t in range(len(self.a_linears)):
            mask = (node_type == t)
            if mask.any():
                final_out[mask] = self.norms[t](self.a_linears[t](aggr_out[mask]))
        return final_out

class ScratchHGTModel(nn.Module):
    def __init__(self, num_types, num_relations, in_dim, hidden_dim, out_dim):
        super().__init__()
        self.input_proj = nn.Linear(in_dim, hidden_dim)
        self.conv1 = ScratchHGTConv(hidden_dim, hidden_dim, num_types, num_relations)
        self.classifier = nn.Linear(hidden_dim, out_dim)

    def forward(self, x, node_type, edge_index, edge_type):
        h = F.gelu(self.input_proj(x))
        h = self.conv1(h, node_type, edge_index, edge_type)
        return self.classifier(h)

# --- RUNNING ---
# Instantiate model with the same parameters
model = ScratchHGTModel(
    num_types=len(all_node_types),
    num_relations=len(rel_map),
    in_dim=128, 
    hidden_dim=128,
    out_dim=dataset.num_classes
).to(device)

model.eval()
with torch.no_grad():
    # Pass the sampled data
    output = model(new_x, new_node_type, final_edge_index, final_edge_type)
    print(f"Success! Sampled Output shape: {output.shape}")

Success! Sampled Output shape: torch.Size([0, 349])
